In [1]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts import FewShotChatMessagePromptTemplate, ChatPromptTemplate, MessagesPlaceholder

chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.1)

movie_few_shot_prompt = FewShotChatMessagePromptTemplate(
  example_prompt=ChatPromptTemplate.from_messages([
    ("human", "Name of the movie is {movie}"),
    ("ai", "Here are 3 emojis that describe the movie: {emojis}")
  ]),
  examples=[
    {"movie": "The Matrix", "emojis": "🎥🤖🌀"},
    {"movie": "Top Gun", "emojis": "🛩️💥🔥"},
    {"movie": "The Godfather", "emojis": "👨‍👨‍👦🔫🍝"}
  ]
)

movie_prompt = ChatPromptTemplate.from_messages([
  ("system", "You are a helpful assistant who summarizes the movie into 3 emojis."),
  movie_few_shot_prompt,
  ("human", "Name of the movie is {movie}")
])

movie_chain = movie_prompt | chat

memory = ConversationSummaryBufferMemory(
  llm=chat, 
  max_token_limit=120,
  memory_key="history",
  return_messages=True
)

def invoke_movie_chain(movie):
  result = movie_chain.invoke({ "movie": movie })
  memory.save_context({"input": movie}, {"output": result.content})

In [2]:
invoke_movie_chain("Oldboy")
invoke_movie_chain("The Social Network")
invoke_movie_chain("Avatar")
memory.load_memory_variables({})["history"]

[HumanMessage(content='Oldboy'),
 AIMessage(content='Here are 3 emojis that describe the movie: 🔄🔪🤯'),
 HumanMessage(content='The Social Network'),
 AIMessage(content='Here are 3 emojis that describe the movie: 💻💰👥'),
 HumanMessage(content='Avatar'),
 AIMessage(content='Here are 3 emojis that describe the movie: 🌿🌌💙')]

In [4]:
from langchain.schema.runnable import RunnablePassthrough

assistant_prompt = ChatPromptTemplate.from_messages([
  ("system", "You are a helpful assistant answers to questions about movies."),
  MessagesPlaceholder(variable_name="history"),
  ("human", "{question}")
])

def load_memory(input):
  return memory.load_memory_variables({})["history"]

assistant_chain = RunnablePassthrough.assign(history=load_memory) | assistant_prompt | chat

assistant_chain.invoke({ "question": "What is the name of the movie that I asked first?"})


AIMessage(content='The name of the movie you asked about first is "Oldboy."')